In [1]:
import pandas as pd
import altair as alt

This is an analysis of Eurostat labour market data, with a focus on groups defined by migration experience

Popular assumptions:
* "It's the market": migrants are underemployed when there are too few jobs for all -> vacancy data;
* Any gaps between migrants and natives are explained by citizenship rights -> comparing country of birth and citizenship data;
* Unfilled vacancies are mainly in the highly regulated sectors (e.g. healthcare), because of stringent criteria -> vacancy data

# 1. Data import and preparation

## 1.1. Employment rate

In [2]:
# Loading the Eurostat CSV file
# Source: https://ec.europa.eu/eurostat/databrowser/view/lfsa_erganedm__custom_8269150/default/table?lang=en

keep_columns = ["citizen", "mgstatus", "geo", "OBS_VALUE"]

# load the data only reading the chosen columns
df = pd.read_csv("data/erganedm_tert_cit.csv", usecols=keep_columns) # adapt to location

df.sample(5)

,citizen,mgstatus,geo,OBS_VALUE
366,TOTAL,NBO,HR,84.3
76,EU27_2020_FOR,TOTAL,RO,NaN
390,TOTAL,TOTAL,ES,81.3
261,NEU27_2020_FOR,FBO,IE,80.7
125,FOR,NBO,LU,78.1


In [3]:
# Convert numeric values, turning missing values into NaN
df["OBS_VALUE"] = pd.to_numeric(df["OBS_VALUE"], errors='coerce')

## 1.2. Overqualification rate

In [4]:
# Loading the Eurostat CSV file
# Source: https://ec.europa.eu/eurostat/databrowser/view/lfsa_eoqgan__custom_8275571/default/table?lang=en

keep_columns = ["citizen", "geo", "OBS_VALUE"]

# load the data only reading the chosen columns
dfo = pd.read_csv("data/eoqgan_overqual_cit.csv", usecols=keep_columns) # adapt to location

dfo.sample(5)

,citizen,geo,OBS_VALUE
112,TOTAL,AT,27.7
83,NAT,SK,23.3
75,NAT,LV,17.7
10,EU27_2020_FOR,EU27_2020,31.8
106,NEU27_2020_FOR,PL,40.9


In [5]:
# Convert numeric values, turning missing values into NaN
dfo["OBS_VALUE"] = pd.to_numeric(df["OBS_VALUE"], errors='coerce')

## 1.3. Vacancy rate

In [6]:
# Loading the Eurostat CSV file
# Source: https://ec.europa.eu/eurostat/databrowser/view/jvs_a_rate_r2__custom_8276608/default/table?lang=en

keep_columns = ["nace_r2", "geo", "OBS_VALUE"]

# load the data only reading the chosen columns
dfv = pd.read_csv("data/vacancy_rate.csv", usecols=keep_columns) # adapt to location

dfv.sample(5)

,nace_r2,geo,OBS_VALUE
20,J,NL,7.6
51,K,SE,3.1
125,Q,PL,0.8
130,Q,SK,0.7
127,Q,RO,1.1


In [7]:
# Convert numeric values, turning missing values into NaN
dfv["OBS_VALUE"] = pd.to_numeric(dfv["OBS_VALUE"], errors='coerce')

In [8]:
dfv.nace_r2.unique()

array(['J', 'K', 'M', 'P', 'Q'], dtype=object)

In [9]:
# Rename categories as the labels are confusing
dfv = dfv.replace(['J', 'K', 'M', 'P', 'Q'],["InfoCom", "Finance", "ProfSciTech", "Education", "HealthSocWork"])

In [10]:
dfv.sample(5)

,nace_r2,geo,OBS_VALUE
124,HealthSocWork,NL,4.2
25,InfoCom,SI,3.4
105,Education,SK,0.2
69,ProfSciTech,IT,2.9
21,InfoCom,PL,3.5


# 2. Calculation of employment gaps

## 2.1. By citizenship -- all foreign nationals

In [11]:
df.citizen.unique() # Display the citizenship categories

array(['EU27_2020_FOR', 'FOR', 'NAT', 'NEU27_2020_FOR', 'TOTAL'],
      dtype=object)

In [12]:
# Creating a pivot table
gapcit = df[df.mgstatus == "TOTAL"].pivot_table('OBS_VALUE', ['geo'], 'citizen')

In [13]:
gapcit["GapAll"] = gapcit["NAT"] - gapcit["FOR"] #native-born minus foreign-born (EU and non-EU)

# The gap is measured in percentage points. Positive values mean native advantage

# Finding countries with the largest gaps
gapcit["GapAll"].sort_values(ascending = False).head(5)

geo
FR    16.1
EL    15.3
DE    15.2
LV    14.4
EE    14.2
Name: GapAll, dtype: float64

## 2.2. By citizenship -- non-EU nationals

In [14]:
gapcit["GapNEU"] = gapcit["NAT"] - gapcit["NEU27_2020_FOR"] # native-born minus non-EU only
# Finding countries with the largest gaps
gapcit["GapNEU"].sort_values(ascending = False).head(5)

geo
DE    22.9
FR    21.8
CY    21.2
NL    20.1
BE    20.1
Name: GapNEU, dtype: float64

## 2.3. By migration status

In [15]:
df.mgstatus.unique() # Display the migration categories

array(['FBO', 'NBO', 'TOTAL'], dtype=object)

In [16]:
# Creating a pivot table
gapmgr = df[df.citizen == "TOTAL"].pivot_table('OBS_VALUE', ['geo'], 'mgstatus')

In [17]:
gapmgr["GapMgr"] = gapmgr["NBO"] - gapmgr["FBO"] # native-born minus foreign-born
# The gap is measured in percentage points. Positive values mean native advantage

# Finding countries with the largest gaps
gapmgr["GapMgr"].sort_values(ascending = False).head(5)

geo
BG    18.7
RO    16.3
LV    14.2
FR    13.2
DE    12.7
Name: GapMgr, dtype: float64

# 3. Calculation of overqualification gaps

## 3.1. All foreign nationals

In [18]:
# Creating a pivot table
gapoq = dfo.pivot_table('OBS_VALUE', ['geo'], 'citizen')

In [19]:
gapoq["GapOQ"] = gapoq["NAT"] - gapoq["FOR"] # native-born minus foreign-born (EU and non-EU)

# The gap is measured in percentage points. Positive values mean native advantage

# Finding countries with the largest gaps
gapoq["GapOQ"].sort_values(ascending = False).head(5)

geo
EL    14.0
IT    11.2
AT     5.7
ES     4.8
BE    -2.7
Name: GapOQ, dtype: float64

## 3.2. Non-EU nationals

In [21]:
gapoq["GapONEU"] = gapoq["NAT"] - gapoq["NEU27_2020_FOR"] # native-born minus non-EU only

# The gap is measured in percentage points. Positive values mean native advantage

# Finding countries with the largest gaps
gapoq["GapONEU"].sort_values(ascending = False).head(5)

geo
EL    15.9
LV    14.0
BE    12.8
HR    11.0
DK     9.9
Name: GapONEU, dtype: float64

In [22]:
data_frames = [gapcit, gapmgr, gapoq]
gapdata = pd.concat(data_frames, join='outer', axis=1)
col_list = ['GapAll','GapNEU','GapMgr','GapOQ', 'GapONEU']
gapdata = gapdata[col_list]

In [23]:
gapdata.sort_values("GapONEU", ascending = False).head(5)

,GapAll,GapNEU,GapMgr,GapOQ,GapONEU
geo,,,,,
EL,15.3,19.6,8.0,14.0,15.9
LV,14.4,15.4,14.2,NaN,14.0
BE,6.0,20.1,6.8,-2.7,12.8
HR,-3.2,3.5,-4.7,NaN,11.0
DK,6.6,8.6,5.9,NaN,9.9


# 4. Vacancy rate statistics

In [24]:
# From the EMN report, identify countries that flagged the qualifications recognition issue
lack_reco = ["AT", "CZ", "DE", "ES", "FI", "IE", "IT", "LU", "LV", "PT", "SE", "SI"] # List from EMN report, p. 15
noreco = pd.DataFrame(columns = ['Country', "LacksRecognition"])
noreco.Country = lack_reco
noreco.LacksRecognition = "Yes"

In [25]:
# Incorporate the recognition issue data into the vacancy dataframe
data = pd.merge(left=dfv, right=noreco,
         left_on="geo", right_on="Country", how="left")
data.sample(5)

,nace_r2,geo,OBS_VALUE,Country,LacksRecognition
55,ProfSciTech,BE,8.3,NaN,NaN
83,Education,BG,0.4,NaN,NaN
23,InfoCom,RO,1.3,NaN,NaN
102,Education,RO,0.4,NaN,NaN
17,InfoCom,LU,3.8,LU,Yes


In [26]:
data['LacksRecognition'].fillna('No', inplace=True)

In [27]:
data = data.drop("Country", axis=1) # time to get rid of the incomplete column from noreco

In [28]:
# Which countries and occupational categories have the highest vacancy rate?
data.sort_values("OBS_VALUE", ascending=False).head(10)

,nace_r2,geo,OBS_VALUE,LacksRecognition
1,InfoCom,BE,8.5,No
55,ProfSciTech,BE,8.3,No
71,ProfSciTech,LU,7.6,Yes
20,InfoCom,NL,7.6,No
0,InfoCom,AT,6.6,Yes
54,ProfSciTech,AT,6.4,Yes
74,ProfSciTech,NL,6.3,No
5,InfoCom,DE,5.7,Yes
58,ProfSciTech,CZ,5.7,Yes
19,InfoCom,MT,5.5,No


In [29]:
# Let's specifically analyse Human health and social work
data[data.nace_r2 == "HealthSocWork"].sort_values("OBS_VALUE", ascending=False).head(10)

,nace_r2,geo,OBS_VALUE,LacksRecognition
111,HealthSocWork,DE,4.3,Yes
124,HealthSocWork,NL,4.2,No
106,HealthSocWork,AT,4.0,Yes
117,HealthSocWork,HU,4.0,No
107,HealthSocWork,BE,3.0,No
115,HealthSocWork,FI,2.9,Yes
128,HealthSocWork,SE,2.8,Yes
122,HealthSocWork,LV,2.7,Yes
116,HealthSocWork,HR,2.5,No
109,HealthSocWork,CY,2.2,No


In [30]:
# Create a pivot table that turns occupations into columns
vac = data.pivot_table('OBS_VALUE', ['geo', 'LacksRecognition'], 'nace_r2')

In [31]:
# Calculate averages for highly regulated and other highly skilled occupations
vac['MeanRg'] = vac[['Education', 'HealthSocWork']].mean(axis=1)
vac ['MeanOther'] = vac[['Finance', 'InfoCom', "ProfSciTech"]].mean(axis=1)

In [32]:
vac.sort_values("MeanRg", ascending=False).head(5)

,nace_r2,Education,Finance,HealthSocWork,InfoCom,ProfSciTech,MeanRg,MeanOther
geo,LacksRecognition,,,,,,,
DE,Yes,3.5,2.4,4.3,5.7,5.2,3.90,4.433333
HU,No,2.4,2.8,4.0,3.4,2.7,3.20,2.966667
NL,No,2.1,5.2,4.2,7.6,6.3,3.15,6.366667
AT,Yes,2.0,3.0,4.0,6.6,6.4,3.00,5.333333
BE,No,2.9,3.7,3.0,8.5,8.3,2.95,6.833333


# 5. Visualise with simple graphs

This is for the general idea -- much better visualisations can be made with other tools

In [33]:
vac.columns

Index(['Education', 'Finance', 'HealthSocWork', 'InfoCom', 'ProfSciTech',
       'MeanRg', 'MeanOther'],
      dtype='object', name='nace_r2')

## Vacancy rate

In [34]:
chart_data = vac.reset_index()
title = alt.TitleParams('Vacancy rate -- highly regulated occupations', anchor='middle')

plot = (
    alt.Chart(chart_data, title=title)
    .mark_bar()
    .encode(
        x=alt.X("geo", sort="y"),
        y="MeanRg"))

plot.encode(color=alt.condition(alt.datum.geo == "EU27_2020", alt.value("red"), alt.value("navy")))

alt.Chart(...)

In [35]:
chart_data = vac.reset_index()
title = alt.TitleParams('Vacancy rate -- other high-skilled occupations', anchor='middle')

plot = (
    alt.Chart(chart_data, title=title)
    .mark_bar()
    .encode(
        x=alt.X("geo", sort="y"),
        y="MeanOther"))

plot.encode(color=alt.condition(alt.datum.geo == "EU27_2020", alt.value("red"), alt.value("navy")))

alt.Chart(...)

## Employment gap by citizenship

In [36]:
gapdata.sample(5)

,GapAll,GapNEU,GapMgr,GapOQ,GapONEU
geo,,,,,
SE,10.0,15.1,8.1,-5.7,-8.9
MT,5.7,9.1,5.2,NaN,NaN
CY,12.4,21.2,11.9,NaN,8.6
BG,NaN,NaN,18.7,-3.2,2.1
SI,13.2,17.0,8.0,NaN,NaN


In [37]:
chart_data = gapdata.reset_index()
title = alt.TitleParams('Employment gap -- Native vs foreign', anchor='middle')

plot = (
    alt.Chart(chart_data, title=title)
    .mark_bar()
    .encode(
        x=alt.X("geo", sort="y"),
        y="GapAll"))

plot.encode(color=alt.condition(alt.datum.geo == "EU27_2020", alt.value("red"), alt.value("navy")))

alt.Chart(...)

In [38]:
chart_data = gapdata.reset_index()
title = alt.TitleParams('Employment gap -- Native vs non-EU', anchor='middle')

plot = (
    alt.Chart(chart_data, title=title)
    .mark_bar()
    .encode(
        x=alt.X("geo", sort="y"),
        y="GapNEU"))

plot.encode(color=alt.condition(alt.datum.geo == "EU27_2020", alt.value("red"), alt.value("navy")))

alt.Chart(...)

In [39]:
chart_data = gapdata.reset_index()
title = alt.TitleParams('Employment gap -- Native vs foreign born', anchor='middle')

plot = (
    alt.Chart(chart_data, title=title)
    .mark_bar()
    .encode(
        x=alt.X("geo", sort="y"),
        y="GapMgr"))

plot.encode(color=alt.condition(alt.datum.geo == "EU27_2020", alt.value("red"), alt.value("navy")))

alt.Chart(...)

In [40]:
chart_data = gapdata.reset_index()
title = alt.TitleParams('Overqualification gap -- Native vs foreign', anchor='middle')

plot = (
    alt.Chart(chart_data, title=title)
    .mark_bar()
    .encode(
        x=alt.X("geo", sort="y"),
        y="GapOQ"))

plot.encode(color=alt.condition(alt.datum.geo == "EU27_2020", alt.value("red"), alt.value("navy")))

alt.Chart(...)

In [41]:
chart_data = gapdata.reset_index()
title = alt.TitleParams('Overqualification gap -- Native vs non-EU', anchor='middle')

plot = (
    alt.Chart(chart_data, title=title)
    .mark_bar()
    .encode(
        x=alt.X("geo", sort="y"),
        y="GapONEU"))

plot.encode(color=alt.condition(alt.datum.geo == "EU27_2020", alt.value("red"), alt.value("navy")))

alt.Chart(...)

# 6. Save the files

In [42]:
gapdata.to_csv("gapsatwork.csv")

In [43]:
vac.to_csv("vacancyrates.csv")